Helper Functions

In [ ]:
import random
# Modified versions of my symbolic aritmetic assignement functions to read two strings instead of filenames
def _add_positive_strings(num1, num2):
    max_len = max(len(num1), len(num2))
    num1 = num1.zfill(max_len)
    num2 = num2.zfill(max_len)

    result = ""
    carry = 0

    for i in range(max_len - 1, -1, -1):
        digit_sum = int(num1[i]) + int(num2[i]) + carry
        carry = digit_sum // 10
        result = str(digit_sum % 10) + result

    if carry > 0:
        result = str(carry) + result

    return result

def double_string(n: str) -> str:
        return _add_positive_strings(n, n)

def mod_exp(base, exp, mod):
    base = int(base)
    exp = int(exp)
    mod = int(mod)
    result = 1
    while exp > 0:
        if exp % 2 == 1: 
            result = (result * base) % mod
        exp //= 2
        base = (base * base) % mod
    return str(result)

def gcd_as_strings(a: str, b: str) -> str:

    while b != "0":
        a, b = b, str(int(a) % int(b))
    return a

def is_coprime_as_strings(a: str, b: str) -> bool:

    return gcd_as_strings(a, b) == "1"

def totient_as_strings(n: str) -> str:

    n_int = int(n)
    count = 0
    for i in range(1, n_int):
        if is_coprime_as_strings(str(i), n):
            count += 1
    return str(count)


def halve_string_number(num: str) -> str:
    if num == "0":
        return "0"

    result = ""
    carry = 0

    for digit in num:
        current = carry * 10 + int(digit)
        result += str(current // 2)
        carry = current % 2

    return result.lstrip("0") or "0"

def double_string(n: str) -> str:
        return _add_positive_strings(n, n)

def multiply_strings(num1: str, num2: str) -> str:
    def add_strings(n1: str, n2: str) -> str:
        return _add_positive_strings(n1, n2)

    x = "0"
    a = num1
    b = num2

    while b != "0":
        if int(b[-1]) % 2 == 1:
            x = add_strings(x, a)
        a = double_string(a)
        b = halve_string_number(b)

    return x

def subtract_strings(num1: str, num2: str) -> str:
    if len(num1) < len(num2) or (len(num1) == len(num2) and num1 < num2):
        num1, num2 = num2, num1
        negative_result = True
    else:
        negative_result = False

    num1, num2 = num1.zfill(max(len(num1), len(num2))), num2.zfill(max(len(num1), len(num2)))

    result = []
    carry = 0

    for i in range(len(num1) - 1, -1, -1):
        digit1 = int(num1[i])
        digit2 = int(num2[i])
        digit1 -= carry
        if digit1 < digit2:
            digit1 += 10
            carry = 1
        else:
            carry = 0

        result_digit = digit1 - digit2
        result.append(str(result_digit))

    while result and result[-1] == '0':
        result.pop()

    final_result = ('-' if negative_result else '') + ''.join(result[::-1])
    return final_result if final_result else '0'

def mod_inverse_as_strings(e: str, phi: str) -> str:
    t, new_t = "0", "1"
    r, new_r = phi, e
    while new_r != "0":
        quotient = str(int(r) // int(new_r))
        t, new_t = new_t, str(int(t) - int(quotient) * int(new_t))
        r, new_r = new_r, str(int(r) - int(quotient) * int(new_r))
    if int(r) > 1:
        raise ValueError("e is not invertible")
    if int(t) < 0:
        t = str(int(t) + int(phi))
    return t

def choose_ephemeral_key(p):
    p_length = len(p)
    ephemeral_key = ['0'] * p_length

    ephemeral_key[0] = str(random.randint(0, int(p[0]) - 1))

    for i in range(1, p_length):
        ephemeral_key[i] = str(random.randint(0, 9))

    return ''.join(ephemeral_key).lstrip('0')

Helper Functions V2

In [ ]:
def safe_prime_number(p):
    p_doubled = double_string(p)
    safe_prime = _add_positive_strings(p_doubled, 1)
    return safe_prime    

def el_gamal_keygen(file_path):
    params = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            params[key.strip()] = value.strip()
    
    g = params.get("g")
    p = params.get("p")
    
    if not g or not p:
        raise ValueError("Parameters file must contain 'g' and 'p'")
    
    private_key = choose_ephemeral_key(p)
    
    public_key = mod_exp(g, private_key, p)
    
    params["private_key"] = private_key
    params["public_key"] = public_key
    
    with open(file_path, 'w') as file:
        for key in ["g", "p", "private_key", "public_key"]:
            file.write(f"{key} = {params[key]}\n")
    
    print("ElGamal Key Generation:")
    print(f"g = {g}, p = {p}")
    print(f"Private Key: {private_key}")
    print(f"Public Key: {public_key}")
    
    return private_key, public_key



def compare_strings(a, b):
    """Compare two numeric strings.
    Returns:
        1 if a > b,
        0 if a == b,
        -1 if a < b
    """
    a = a.lstrip('0') or '0'
    b = b.lstrip('0') or '0'
    if len(a) > len(b):
        return 1
    elif len(a) < len(b):
        return -1
    else:
        if a > b:
            return 1
        elif a < b:
            return -1
        else:
            return 0

def mod_strings(a, b):
    """Compute a mod b where a and b are numeric strings."""
    while compare_strings(a, b) >= 0:
        a = subtract_strings(a, b)
    return a

def multiply_mod(a, b, mod):
    """Compute (a * b) mod mod where a, b, mod are numeric strings."""
    product = multiply_strings(a, b)
    return mod_strings(product, mod)

Diffie-Hellman

In [ ]:
def diffie_hellman_key_exchange(file_path):

    params = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            params[key.strip()] = value.strip()
    
    g = params["g"] 
    p = params["p"]  
    a = choose_ephemeral_key(p)
    b = choose_ephemeral_key(p)

    A = mod_exp(g, a, p)  
    B = mod_exp(g, b, p)  
    shared_secret_A = mod_exp(B, a, p) 
    shared_secret_B = mod_exp(A, b, p)  

    print("Parameters:")
    print(f"g = {g}, p = {p}, a = {a}, b = {b}")
    print("Public Keys:")
    print(f"User A Public Key (A): {A}")
    print(f"User B Public Key (B): {B}")
    print("Shared Secret Keys:")
    print(f"Shared Secret computed by User A: {shared_secret_A}")
    print(f"Shared Secret computed by User B: {shared_secret_B}")

    assert shared_secret_A == shared_secret_B, "Shared keys do not match!"
    print("Key Exchange Successful. Shared Secret:", shared_secret_A)
        
            

Launch Diffie-Hellman

In [6]:
file_path = "parameters.txt"
diffie_hellman_key_exchange(file_path)

Parameters:
g = 5, p = 5471619276639877320977, a = 722897160152761146180, b = 3447033392577157977618
Public Keys:
User A Public Key (A): 1015321453020607120417
User B Public Key (B): 1166478526748459937233
Shared Secret Keys:
Shared Secret computed by User A: 4307203896206054585436
Shared Secret computed by User B: 4307203896206054585436
Key Exchange Successful. Shared Secret: 4307203896206054585436


El Gamal

In [ ]:
def el_gamal_encryption(file_path, message):
    params = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            params[key.strip()] = value.strip()
    
    g = params["g"]
    p = params["p"]
    y = params["public_key"]
    k = choose_ephemeral_key(p)
    
    r = mod_exp(g, k, p)
    shared_key = mod_exp(y, k, p)
    c = multiply_mod(message, shared_key, p)
    
    print("ElGamal Encryption:")
    print(f"g = {g}, p = {p}, y (public_key) = {y}")
    print(f"Ephemeral Key (k): {k}")
    print(f"Ciphertext: (r, c) = ({r}, {c})")
    
    return r, c

def el_gamal_decryption(file_path, r, c):
    params = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            params[key.strip()] = value.strip()
    
    p = params["p"]
    x = params["private_key"]
    
    shared_key = mod_exp(r, x, p)
    shared_key_inv = mod_inverse_as_strings(shared_key, p)
    m = multiply_mod(c, shared_key_inv, p)
    
    print("ElGamal Decryption:")
    print(f"Shared Key: {shared_key}")
    print(f"Decrypted Message: {m}")
    
    return m

In [ ]:
def initialize_parameters(file_path, g, p):
    with open(file_path, 'w') as file:
        file.write(f"g = {g}\n")
        file.write(f"p = {p}\n")

# Example Initialization
initialize_parameters('parameters1.txt', '5', '56775307')

private_key, public_key = el_gamal_keygen('parameters1.txt')
message = "2112321"  

r, c = el_gamal_encryption('parameters1.txt', message)
decrypted_message = el_gamal_decryption('parameters1.txt', r, c)
print(f"Original Message: {message}")
print(f"Decrypted Message: {decrypted_message}")


ElGamal Key Generation:
g = 5, p = 56775307
Private Key: 20140040
Public Key: 54484027
ElGamal Encryption:
g = 5, p = 56775307, y (public_key) = 54484027
Ephemeral Key (k): 20486919
Ciphertext: (r, c) = (10180595, 23347638)
ElGamal Decryption:
Shared Key: 3170801
Decrypted Message: 2112321
Original Message: 2112321
Decrypted Message: 2112321


RSA


In [ ]:
BASE64_CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"

def base64_encode(input_str):
    """
    Manually encodes a string into Base64.
    """
    binary_str = ''.join([format(ord(c), '08b') for c in input_str])
    padding_length = (6 - len(binary_str) % 6) % 6
    binary_str += '0' * padding_length
    chunks = [binary_str[i:i+6] for i in range(0, len(binary_str), 6)]

    encoded = ''.join([BASE64_CHARS[int(chunk, 2)] for chunk in chunks])
    padding_chars = (4 - len(encoded) % 4) % 4
    encoded += '=' * padding_chars
    
    return encoded

def base64_decode(encoded_str):
    padding = encoded_str.count('=')
    encoded_str = encoded_str.rstrip('=')
    binary_str = ''.join([format(BASE64_CHARS.index(c), '06b') for c in encoded_str])
    if padding:
        binary_str = binary_str[:-padding*2]

    bytes_chunks = [binary_str[i:i+8] for i in range(0, len(binary_str), 8)]
    decoded = ''.join([chr(int(byte, 2)) for byte in bytes_chunks if len(byte) == 8])
    
    return decoded


def rsa_encrypt_as_strings(plaintext: str, n: str, e: str) -> str:
    block_size = (len(n) - 1) // 3  
    blocks = [plaintext[i:i+block_size] for i in range(0, len(plaintext), block_size)]
    encrypted_blocks = []
    for block in blocks:
        m = int.from_bytes(block.encode('utf-8'), 'big')
        if m >= int(n):
            raise ValueError("Block too large for the key size")
        c = mod_exp(m, e, n)
        encrypted_blocks.append(c)
    return ','.join(encrypted_blocks)

def rsa_decrypt_as_strings(ciphertext: str, n: str, d: str) -> str:
    encrypted_blocks = ciphertext.split(',')
    decrypted_message = []
    for block in encrypted_blocks:
        m = int(mod_exp(block, d, n))
        num_bytes = (m.bit_length() + 7) // 8
        decrypted_block = m.to_bytes(num_bytes, 'big').decode('utf-8', errors='ignore')
        decrypted_message.append(decrypted_block)
    return ''.join(decrypted_message)

def encrypt_multilingual(plaintext: str, n: str, e: str) -> str:
    """
    Encrypts multilingual plaintext using RSA by first Base64 encoding it.
    """
    b64_text = base64_encode(plaintext)
    
    ciphertext = rsa_encrypt_as_strings(b64_text, n, e)
    return ciphertext

def decrypt_multilingual(ciphertext: str, n: str, d: str) -> str:
    b64_text = rsa_decrypt_as_strings(ciphertext, n, d)
    plaintext = base64_decode(b64_text)
    return plaintext

# RSA Key Generation
def generate_rsa_keys(p: str, q: str, e: str):
    """
    Generates RSA keys (n, e, d) using the given p, q, and e.
    Calculates \u03d5(n) and checks if e is coprime with \u03d5(n) using gcd_as_strings.
    """
    n = multiply_strings(p, q)
    subtracted_p = subtract_strings(p, "1")
    subtracted_q = subtract_strings(q, "1")
    phi = multiply_strings(subtracted_p, subtracted_q)
    if gcd_as_strings(e, phi) != "1":
        raise ValueError(f"e ({e}) is not coprime with \u03d5(n) ({phi})")
    d = mod_inverse_as_strings(e, phi)
    return n, e, d

# Reading RSA Parameters from File
def read_rsa_parameters(file_path):
    """
    Reads RSA parameters p, q, e from a given file.
    """
    params = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            params[key.strip()] = value.strip()
    
    p = params.get("p")
    q = params.get("q")
    e = params.get("e")
    
    if not p or not q or not e:
        raise ValueError("The file must contain 'p', 'q', and 'e' parameters.")
    
    return p, q, e

In [ ]:
if __name__ == "__main__":
    file_path = "parameters2.txt"
    p, q, e = read_rsa_parameters(file_path)

    try:
        n, e, d = generate_rsa_keys(p, q, e)
        print(f"Public Key: (n={n}, e={e})")
        print(f"Private Key: (n={n}, d={d})")

        multilingual_text = "Motörhead is my favorite band!"
        print(f"Original Message: {multilingual_text}")
        ciphertext = encrypt_multilingual(multilingual_text, n, e)
        print(f"Ciphertext: {ciphertext}")

        decrypted_text = decrypt_multilingual(ciphertext, n, d)
        print(f"Decrypted Message: {decrypted_text}")

    except ValueError as ve:
        print(f"Error: {ve}")

Public Key: (n=22180334752531629071, e=10009)
Private Key: (n=22180334752531629071, d=16183733101727265073)
Original Message: Motörhead is my favorite band!
Ciphertext: 5064062055049873055,13193440876067754786,13199711043469832295,11364479636950397837,14112716701533511549,13443012065312276876,5614918658898817950
Decrypted Message: Motörhead is my favorite band!
